# Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей.
---
## Задание:
1.   Выберите набор данных (датасет) для решения задачи классификации или регрессии.

2.   В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.

3.  С использованием метода train_test_split разделите выборку на обучающую и тестовую.

4.  Обучите модель ближайших соседей для произвольно заданного гиперпараметра K. Оцените качество модели с помощью подходящих для задачи метрик.

5.  Произведите подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV и кросс-валидации, оцените качество оптимальной модели. Используйте не менее двух стратегий кросс-валидации.

6.  Сравните метрики качества исходной и оптимальной моделей.












## Загрузка датасета и чтение с помощью библиотеки Pandas и предварительная обработка данных

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1

data = pd.read_csv('study_performance.csv')
# Для числовых столбцов можно использовать среднее значение
data['math_score'] = data['math_score'].fillna(data['math_score'].mean())
# Удаление дубликатов
data = data.drop_duplicates()




## Кодирование категориальных признаков

In [7]:
for column in ['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])

## Разделим выборку на обучающую и тестовую.

In [8]:
X = data.drop(['math_score', 'reading_score','writing_score'], axis=1)
y = data['math_score']  # Прогнозируем математический балл, как пример
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучим модель ближайших соседей для произвольно заданного гиперпараметра K. Оценим качество модели с помощью подходящих для задачи метрик.

In [21]:
k = 5  # Произвольное значение К
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, predictions)}")


Accuracy: 0.005


## Произведем подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV и кросс-валидации, оценим качество оптимальной модели. Используем не менее двух стратегий кросс-валидации.


In [22]:
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 13]}
kf = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=kf, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(f"GridSearchCV Best params: {grid_search.best_params_}, Best score: {grid_search.best_score_}")

# RandomizedSearchCV
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_grid, n_iter=3, cv=skf, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print(f"RandomizedSearchCV Best params: {random_search.best_params_}, Best score: {random_search.best_score_}")

GridSearchCV Best params: {'n_neighbors': 3}, Best score: 0.015


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV Best params: {'n_neighbors': 3}, Best score: 0.02
